In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import holodeck as holo
import holodeck.plot
import holodeck.host_relations
from holodeck.constants import *

## Sigmoid Bulge-Fraction

### Analytic

In [ ]:
def sigmoid(xx, frac_lo=1.0, frac_hi=1.0, width=2.0):
    steep = 1.0 / width
    yy = frac_lo + (frac_hi - frac_lo) / (1.0 + ((1.0 / xx) - 1.0)**steep)
    yy[(xx >= 1.0) | (yy >= frac_hi)] = frac_hi
    return yy

# def inverse(yy, frac_lo=1.0, frac_hi=1.0, width=2.0):
#     sel = (yy < 1.0)
#     xx = np.ones_like(yy)

#     zz = (frac_hi - yy[sel]) / (yy[sel] - frac_lo)
#     xx[sel] = (np.power(yy, width) - 1.0) ** -1
#     xx[~sel] =

#     return xx

def deriv(xx, frac_lo=1.0, frac_hi=1.0, width=2.0):
    k = 1.0 / width
    fl = frac_lo
    fh = frac_hi
    yy = sigmoid(xx, frac_lo=frac_lo, frac_hi=frac_hi, width=width)

    yp = (fh - fl) / np.square(1.0 + np.power(1/xx - 1.0, k))
    yp *= k * np.power(1/xx - 1, k-1)
    yp *= np.power(xx, -2)
    yp[(xx >= 1.0) | (yy >= frac_hi)] = 0.0

    return yp

xx = np.logspace(-4, 1, 100)

In [ ]:
frac_hi = 0.5
low_fractions = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
low_fractions = np.array(low_fractions)

fig, ax = holo.plot.figax(yscale='lin')
limits = []
for flo in low_fractions:
    yy = sigmoid(xx, frac_lo=flo, frac_hi=frac_hi)
    cc, = ax.plot(xx, yy, label=f'{flo:.2f}')
    cc = cc.get_color()
    ax.axhline(flo, alpha=0.75, ls='--', lw=0.5, color=cc)
    limits.append(yy[0])

ax.legend()

plt.show()

In [ ]:
widths = [0.5, 1.0, 2.0, 3.0]

fig, ax = holo.plot.figax(yscale='lin')
limits = []
for wid in widths:
    yy = sigmoid(xx, frac_lo=0.5, frac_hi=0.8, width=wid)
    cc, = ax.plot(xx, yy, label=f'{wid:.2f}')
    ax.axvline(10**-wid, color=cc.get_color(), ls='--', alpha=0.5)

ax.legend()
plt.show()

In [ ]:
frac_hi = 0.5
width = 1.5
# low_fractions = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
low_fractions = [0.05, 0.1, 0.2, 0.5, 0.8]
low_fractions = np.array(low_fractions)

fig, ax = holo.plot.figax(yscale='lin')
fig, tw = holo.plot.figax(yscale='log')
limits = []
for flo in low_fractions:
    kw = dict(frac_lo=flo, frac_hi=frac_hi, width=width)
    # yy = sigmoid(xx, **kw)
    # cc, = ax.plot(xx, yy, label=f'{flo:.2f}')
    # cc = cc.get_color()

    yp = deriv(xx, **kw)
    xl = xx * (1.0 - 1e-2)
    xh = xx * (1.0 + 1e-2)
    yl = sigmoid(xl, **kw)
    yh = sigmoid(xh, **kw)
    dd = (yh - yl) / (xh - xl)
    err = (yp - dd) / dd

    cc, = ax.plot(xx, yp, ls='--', label=f"{flo:.2f}")
    # cc = None
    cc = cc.get_color()
    ax.plot(xx, dd, ls='-', alpha=0.5, color=cc)
    tw.plot(xx, np.fabs(err), alpha=0.5)


ax.legend()

plt.show()

### Implementation

In [ ]:
frac_lo = 0.3
frac_hi = 0.9
mchar = 10.5
width = 1.0
bf = holo.host_relations.BF_Sigmoid(
    bulge_frac_lo=frac_lo, bulge_frac_hi=frac_hi, mstar_char_log10=mchar, width_dex=width
)

In [ ]:
ms = np.logspace(8, 12, 100) * MSOL
mb = bf.mbulge_from_mstar(ms)

ms_test = bf.mstar_from_mbulge(mb)

fig, ax = holo.plot.figax()
ax.plot(ms/MSOL, mb/MSOL, '.-')
ax.plot(ms_test/MSOL, mb/MSOL, 'x-', alpha=0.2)

fig, ax = holo.plot.figax()
err = np.fabs((ms_test - ms) / ms)
ax.plot(ms/MSOL, err, '.-')

plt.show()

### Derivative: ``dmstar_dmbulge``

In [ ]:
bf = holo.host_relations.BF_Sigmoid(
    bulge_frac_lo=frac_lo, bulge_frac_hi=frac_hi, mstar_char_log10=mchar, width_dex=width
)

ms = np.logspace(8, 12, 1000) * MSOL
# use forward relationship
mb = bf.mbulge_from_mstar(ms)
# get derivatives (numerical, internally)
dms_dmb = bf.dmstar_dmbulge(mb)

delta = 1.0e-4
ms_hi = ms * (1.0 + delta/2.0)
ms_lo = ms * (1.0 - delta/2.0)
mb_hi = bf.mbulge_from_mstar(ms_hi)
mb_lo = bf.mbulge_from_mstar(ms_lo)
dms_dmb_numeric = (ms_hi - ms_lo) / (mb_hi - mb_lo)
error = np.fabs(dms_dmb - dms_dmb_numeric) / dms_dmb_numeric
print(holo.utils.stats(error))

fig, axes = holo.plot.figax(figsize=[10, 8], nrows=2)
axes[0].plot(mb/MSOL, dms_dmb/MSOL, '.-')
axes[0].plot(mb/MSOL, dms_dmb_numeric/MSOL, 'x-', alpha=0.2)

axes[1].plot(mb/MSOL, error, '.-')

plt.show()